In [ ]:
import pandas as pd

import matplotlib.pylab as plt
import numpy as np

In [ ]:
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
topdir= "./"

filename = 'DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000_BNC.pkl'

df_sp = pd.read_parquet(f"{topdir}{filename}")

In [ ]:
df_sp

In [ ]:
df_sp.info()

In [ ]:
n_org= 100000
mask = df_sp["spmode"]== "-999"
num= len(df_sp[mask])
num

In [ ]:
df_sp[mask]

In [ ]:
len(df_sp)

In [ ]:
df_sp['spmode'].value_counts()

In [ ]:
def sig_efficiency(df,thresh,BNC= False, orig_num=None):
    
    
    mask = df["spmode"]== "-999"
    
    df_sig= df[mask]

    if orig_num is None:
        print("Please pass in the original number of events!")
        return None
    
    num_reco= len(df_sig)

    eff_reco = num_reco / orig_num
    
    cm1= df_sig["cut_-1"]== True
    cm2= df_sig["cut_2"]== True
    cm3= df_sig["cut_3"]== True
    cm4= df_sig["cut_4"]== True
    cm6= df_sig["cut_6"]== True

    not_used_in_training = df_sig['used_in_sig_train']

    num_reco_not_training = len(df_sig[not_used_in_training])
    
    thresh_mask= df_sig["proba"]>= thresh

    mask = thresh_mask & not_used_in_training
    if BNC:
        mask = mask & cm2 & cm3 & cm4# & cm6
    else:
        mask = mask & cm1

    n_reco_not_training_pass_selection_criteria= len(df_sig[mask])
    
    eff_sc= n_reco_not_training_pass_selection_criteria/num_reco_not_training
    
    return eff_reco, eff_sc, eff_reco*eff_sc

In [ ]:
er, esc, etot = sig_efficiency(df_sp, 0.97, BNC= True, orig_num=100000)

print(f"eff (reco): {100*er:.2f}%")
print(f"eff (sc):   {100*esc:.2f}%")
print(f"eff (tot):  {100*etot:.2f}%")



In [ ]:
xpts = []
ypts = []
for i in np.linspace(0,1,100):
    er, esc, etot = sig_efficiency(df_sp, i, BNC= True, orig_num=100000)
    xpts.append(i)
    ypts.append(etot)
    



In [ ]:
plt.figure()
plt.plot(xpts, ypts)

# Playing around with uncertainies

In [ ]:
k = 90
N = 900

eff = k/N

eff_unc = (1/N)*np.sqrt(k*(1-k/N))

print(f"{100*eff:.4f} +/- {100*eff_unc:.4f}   ({eff_unc/eff:.4f} frac uncert)")

# Calculate the branching fraction

Given the number of observed events, and the threshold cut, calculate the branching fraction.

In [ ]:
nB= 460e6

n_obs= 3.5

eff = 0.07

Br_lam_p_pi = 0.641 # 64.1% +/- 0.5

Br_B_p_lam= n_obs / (nB * eff * Br_lam_p_pi) 

print(f"Br_B_p_lam: {Br_B_p_lam:.3e}")

In [ ]:
#n_obs= nB * Br_B_p_lam * eff * Br_lam_p_pi


# We want this
#Br_B_p_lam= n_obs / (nB * eff * Br_lam_p_pi) 